In [18]:
SYSTEM_PROMPT = """
당신은 'CS 커뮤니케이션 전문 AI'입니다. 
사용자의 입력에 포함된 욕설, 비하 발언, 과격한 감정 표현을 모두 제거하고, 
그 안에 담긴 '핵심 요구사항'과 '불편의 원인'만을 추출하여 
가장 정중하고 격식 있는 비즈니스 언어(하십시오체)로 재작성하십시오.

[변환 규칙]
1. 욕설/비속어는 절대 출력에 포함하지 말 것.
2. 화자의 감정은 "매우 유감스럽게 생각하며", "강력하게 요청합니다" 등으로 순화할 것.
3. 문장은 간결하고 명확하게 작성할 것.

### 변환 예시:
- 입력: "아니 도대체 배송 언제 오냐고! 사람 장난해? 당장 환불해줘!!"
- 변환: "배송이 언제 올까요? 즉각적인 환불 조치를 요청하겠습니다."

- 입력: "너 이름 뭐야? 왜 이렇게 서비스가 엉망이야? 진짜 짜증나!"
- 변환: "당신의 이름은 무엇인가요? 서비스 품질에 대해 불만이 있습니다."
"""


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name = "K-intelligence/Midm-2.0-Mini-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="cpu"
)
model = torch.compile(model)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
def stream_neutralize(user_input):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_input}
    ]
    
    # 템플릿 적용 및 텐서 변환 (MPS로 이동)
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cpu")

    # 스트리머 설정 (출력되는 토큰을 즉시 화면에 뿌려줌)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    print("\n" + "="*50)
    print(f"🗣️ 원문: {user_input}")
    print("🤖 순화: ", end="", flush=True)

    # 생성 시작
    generation_kwargs = dict(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=64,
        do_sample=True,     # 자연스러운 문장을 위해 샘플링 사용
        temperature=0.3,    # 차분한 톤 유지를 위해 온도 낮춤
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id
    )

    # 생성 (스트리머가 내부적으로 출력하므로 변수에 받을 필요 없음)
    model.generate(**generation_kwargs)
    print("\n" + "="*50 + "\n")

In [21]:
prompt = "야 이 씨발새끼야! 내가 그러지 말랬잖아!!"     
stream_neutralize(prompt)


🗣️ 원문: 야 이 씨발새끼야! 내가 그러지 말랬잖아!!
🤖 순화: 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 